In [10]:
require 'chainer'
require 'fileutils'
require 'optparse'
require 'tmpdir'

false

In [11]:
class MLP < Chainer::Chain
  L = Chainer::Links::Connection::Linear
  R = Chainer::Functions::Activation::Relu

  def initialize(n_units, n_out)
    super()
    init_scope do
      @l1 = L.new(nil, out_size: n_units)
      @l2 = L.new(nil, out_size: n_units)
      @l3 = L.new(nil, out_size: n_out)
    end
  end

  def call(x)
    h1 = R.relu(@l1.(x))
    h2 = R.relu(@l2.(h1))
    @l3.(h2)
  end
end

:call

In [12]:
args = {
  batchsize: 100,
  frequency: -1,
  epoch: 20,
  resume: nil,
  unit: 1000,
  out: 'result'
}

{:batchsize=>100, :frequency=>-1, :epoch=>20, :resume=>nil, :unit=>1000, :out=>"result"}

In [13]:
model = Chainer::Links::Model::Classifier.new(MLP.new(args[:unit], 10))

optimizer = Chainer::Optimizers::Adam.new
optimizer.setup(model)
train, test = Chainer::Datasets::Mnist.get_mnist

train_iter = Chainer::Iterators::SerialIterator.new(train, args[:batchsize])
test_iter = Chainer::Iterators::SerialIterator.new(test, args[:batchsize], repeat: false, shuffle: false)

updater = Chainer::Training::StandardUpdater.new(train_iter, optimizer, device: -1)
trainer = Chainer::Training::Trainer.new(updater, stop_trigger: [args[:epoch], 'epoch'], out: args[:out])

trainer.extend(Chainer::Training::Extensions::Evaluator.new(test_iter, model, device: -1))

# Take a snapshot for each specified epoch
frequency = args[:frequency] == -1 ? args[:epoch] : [1, args[:frequency]].max
trainer.extend(Chainer::Training::Extensions::Snapshot.new, trigger: [frequency, 'epoch'], priority: -100)

trainer.extend(Chainer::Training::Extensions::LogReport.new)
trainer.extend(Chainer::Training::Extensions::PrintReport.new(['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))
trainer.extend(Chainer::Training::Extensions::ProgressBar.new)

#<Chainer::Training::ExtensionEntry:0x000055853c010678 @extension=#<Chainer::Training::Extensions::ProgressBar:0x000055853c010a60 @training_length=nil, @status_template=nil, @update_interval=100, @bar_length=50, @out=#<IO:<STDOUT>>, @recent_timing=[], @name="Chainer::Training::Extensions::ProgressBar">, @trigger=#<Chainer::Training::Triggers::IntervalTrigger:0x000055853c0108d0 @period=1, @unit="iteration", @count=0, @previous_iteration=0, @previous_epoch_detail=0.0>, @invoke_before_training=false, @priority=100>

In [14]:
if args[:resume]
  Chainer::Serializers::MarshalDeserializer.load_file(args[:resume], trainer)
end

In [15]:
trainer.run

true